In [1]:
import os
import spacy
import json
from tqdm import tqdm
import numpy as np
from scipy.special import softmax

In [2]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, AutoConfig

In [3]:
MODEL="cardiffnlp/twitter-roberta-base-sentiment-latest"

In [4]:
nlp = spacy.load("en_core_web_sm")

In [5]:
def process_file(filename):
    refined_json = dict()
    #sentiment_task = pipeline("sentiment-analysis", model=MODEL, tokenizer=MODEL)
    tokenizer = AutoTokenizer.from_pretrained(MODEL)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL)
    config = AutoConfig.from_pretrained(MODEL)
    with open(filename, "r") as f:
        content = json.load(f)

    for k, v in content.items():
        c = " ".join(v.keys())
        doc = nlp(c)
        for word in doc:
            if word.is_stop or word.is_punct or word.is_space or word.like_num:
                txt = word.text
                del v[txt]
        refined_json[k] = v
    
    
    for k, v in refined_json.items():
        for term, frequency in tqdm(v.items()):
            info = dict()
            v[term] = dict()
            v[term]["frequency"] = frequency
#             result = sentiment_task(term)[0]
#             v[term]["sentiment"] = result
            token = tokenizer(term, return_tensors='pt')
            #print(token)
            output = model(**token)
            scores = output[0][0].detach().numpy()
            scores = softmax(scores)
            ranking = np.argsort(scores)
            ranking = ranking[::-1]
            v[term]["sentiment"] = dict()
            for i in range(scores.shape[0]):
                l = config.id2label[ranking[i]]
                s = round(float(scores[ranking[i]]), 3)
                v[term]["sentiment"][l] = s
                
            

    new_name = "refined_" + filename
    with open(new_name, "w") as f:
        json.dump(refined_json, f, indent=4)
    

In [6]:
process_file("results_3.json")

C:\Users\jaeyo\anaconda3\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model)

In [7]:
process_file("results_0.json")
process_file("results_1.json")
process_file("results_2.json")

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 363/363 [00:07<00:00, 51.08it/s]
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected 